**Dependencies**

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Dependencies**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import csv
from csv import writer
from transformers import AutoTokenizer, T5ForConditionalGeneration
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import difflib
from gpt import GPT
from gpt import Example
import openai
import Column_Schema
import json
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import warnings 
warnings.filterwarnings("ignore") 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Using T5 Fine-tuned model on Wiki-SQL**

In [ ]:
model_name = "mrm8488/t5-small-finetuned-wikiSQL" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


**Generating SQL query from T5 Finetuned**

In [ ]:
def get_sql(query):
  input_text = "translante English to SQL: %s </s>" % query
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
def Append_csv_set(query, csvfile):
  SQL_T5=get_sql(query)
  dataframe=[query, SQL_T5]
  with open(csvfile, 'a') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(dataframe)
    f_object.close()

**Fine Tune GPT3**

**For generating .jsonl file for finetunning**
Note:do not run until retuning the model



In [ ]:
#!yes | openai tools fine_tunes.prepare_data -f 'Fine_tune_dataset.csv'

**Opening Training file** 
Note:do not run until retuning the model

In [ ]:
with open("Data_Sample_prepared.jsonl") as f:
      response = openai.File.create(file=f, purpose='fine-tune')
print(response)

{
  "bytes": 153363,
  "created_at": 1665216795,
  "filename": "file",
  "id": "file-nhpa60zOsePlZAZHuTIT2bqF",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


**Creating Finetune Instances**
Note:do not run until retuning the model

In [ ]:
response = openai.FineTune.create(training_file="file-nhpa60zOsePlZAZHuTIT2bqF", model='ada')
print(response)

{
  "created_at": 1665217306,
  "events": [
    {
      "created_at": 1665217306,
      "level": "info",
      "message": "Created fine-tune: ft-5WYz8JMe8s5nmR2f6KwhDIso",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-5WYz8JMe8s5nmR2f6KwhDIso",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-RfTVRwQZuDLbPS2BYvPjPffU",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 153363,
      "created_at": 1665216795,
      "filename": "file",
      "id": "file-nhpa60zOsePlZAZHuTIT2bqF",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1665217306,
  "validation_files": []
}


**Retrieving FineTune Information** Note:do not run until retuning the model

In [ ]:
response = openai.FineTune.retrieve(id="id")
print(response)

{
  "created_at": 1665216838,
  "events": [
    {
      "created_at": 1665216838,
      "level": "info",
      "message": "Created fine-tune: ft-QLRPwoN2jB3iyOssoYwOoOaF",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-QLRPwoN2jB3iyOssoYwOoOaF",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-RfTVRwQZuDLbPS2BYvPjPffU",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 153363,
      "created_at": 1665216795,
      "filename": "file",
      "id": "file-nhpa60zOsePlZAZHuTIT2bqF",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1665216838,
  "validation_files": []
}


**Getting Fineout outputs**

In [ ]:
def gpt3_classifier(item, fine_tuned_model, is_log=False):
    
  result = openai.Completion.create(model=fine_tuned_model, 
                                    prompt=(item))['choices'][0]['text'] 

  result = result.split('\n')
  result=result[0]

  return result

**Main function**

In [ ]:
def main(query):
  Append_csv_set(query,'Data_Sample.csv')
  result=gpt3_classifier(query + ' -->', 'ada:ft-student-2022-10-08-09-13-01')
  return result

**Query Input**

In [ ]:
query='What is the value of LEI_Code where Amount is 1000'
query, res1=Column_Schema.column_name_pred(query,'/content/data_test.csv')

**Output**

In [1]:
%env OPENAI_API_KEY = yourkey

env: OPENAI_API_KEY=yourkey


In [ ]:
SQL_response=main(query)
SQL_response, res2=Column_Schema.column_name_pred(SQL_response,'/content/data_test.csv')

In [ ]:
SQL_response

'select LEI_Code from table where amount = 1000'